In [139]:
from PyPDF2 import PdfReader
from PIL import Image
import pytesseract
import fitz

## Extract text from PDF

- In this step I will extract text from the PDF file.
- Here I assume that all the given resumes are native PDFs.
- The diffrence is that a natve PDF has a PDF extension but it can also be an image PDF which will need a diffrent approach.

## Edit the file path accordingly
- Make sure that this notebook and your resume are in the same directory.
- Replace "Anusha Agarwal Resume.pdf" with the filename of your resume.

In [140]:
file_path = "Anusha Agarwal Resume.pdf"

In [141]:
reader = PdfReader(file_path)

pdf_string_1 = ''

for page in range(len(reader.pages)):
    pdf_string_1 += reader.pages[page].extract_text()

#Text stored in pdf_string_1

#This will store all the extracted information
information = {}

import re
#define the regex pattern
number_pattern = r"\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"
email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"

number_match = re.search(number_pattern, pdf_string_1)
email_match = re.search(email_pattern, pdf_string_1)

if number_match:
    phone_no = number_match.group()
    information['Phone No'] = phone_no

if email_match:
    email = email_match.group()
    information['Email'] = email

## Now we need to extract the name of the person
- This will require some NLP as name does not follow any perticular format except that it is 2 proper nouns one after another
- We will use pretrained models from spacy
- To download this model, we use "!python -m spacy download en_core_web_lg"
- This is one-time download and will not slow down the performance of this code.
- If the system already has this model, this step can be skipped entirely

In [142]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [49]:
!python -m spacy download en_core_web_lg

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 393.8 kB/s eta 0:00:33
     ---------------------------------------- 0.1/12.8 MB 1.1 MB/s eta 0:00:12
      --------------------------------------- 0.3/12.8 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.7/12.8 MB 3.2 MB/s eta 0:00:04
     ----- ---------------------------------- 1.7/12.8 MB 6.3 MB/s eta 0:00:02
     ------ --------------------------------- 2.1/12.8 MB 7.3 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 6.6 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 6.6 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/12.8 MB 6.3 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 6.3 MB/s eta 0:00:02
     ---------- ----------------------------- 3.4/12.8 MB 6.4 MB/


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [143]:
#nlp_1 = spacy.load('en_core_web_sm')
nlp_2 = spacy.load('en_core_web_lg')
from spacy.matcher import Matcher

In [144]:
#sample_1 = nlp_1(pdf_string_1)
sample_2 = nlp_2(pdf_string_1)

def extract_name(resume_text):
    sample_2 = nlp_2(resume_text)

    matcher = Matcher(nlp_2.vocab)
    
    # First name and Last name are always Proper Nouns
    pattern = [
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],
        [{'POS': 'PROPN'}, {'POS': 'PROPN'},{'POS': 'PROPN'}],
              ]
    
    matcher.add('NAME',pattern)
    
    matches = matcher(sample_2)
    
    for match_id, start, end in matches:
        span = sample_2[start:end]
        return span.text

result_in_pdf = extract_name(pdf_string_1)
information['Name'] = result

In [145]:
information

{'Phone No': '8121791433',
 'Email': 'anusha.agarwal564@gmail.com',
 'Name': 'Anusha Agarwal'}